In [1]:
#!pip install transformers
from transformers import pipeline
import torch

classifier = pipeline("zero-shot-classification", model='NbAiLab/nb-bert-base-mnli')


In [2]:
sequences = ['Copper Inside er importert av Dui Clinique AS, distribuert videre til andre aktører og solgt i flere nettbutikker i Norge og andre europeiske land. Utstyret inneholder dikobberoksid, og er markedsført som «gjenbrukbare kobbermunnbind».',
'Kom til VM med ett mål og det var medalje på sprinten. Det klarte hun mot alle odds. Sesongen har vært fryktelig vanskelig, men på en nesten utrolig måte klarte hun å toppe formen til klassisk sprinten.',
'Lagets yngste tok ansvar som ankerkvinne og sikret stafettgull. Leverer to meget gode renn individuelt også.',
'Andelen nordmenn som sier de tror på Gud, har falt fra 53 prosent til 30 prosent på 35 år. Human-Etisk Forbund mener norsk politikk er i utakt med befolkningen.',
'Andelen som ser på seg selv som personlig kristne, har gått ned fra 24 prosent til 21 prosent de siste to åra. På lang sikt har imidlertid denne andelen imidlertid holdt seg stabil og er nå på nivået med målingene på midten av 1980-tallet.',
'Unge fortvilte mennesker, mange med selvmordstanker, har det siste året tatt kontakt med Mental Helse. Pågangen har vært så stor at organisasjonen bare klarer å besvare halvparten av alle som ringer.',
'Forskning viser at ryggsmerter kan være et tidlig symptom på coronaviruset.',
'Det var etter at Ødegaard ble utelatt fra kamptroppen i Real Madrids cupkamp mot Alcoyano at ryktene virkelig begynte å røre på seg.']

In [ ]:
#The default hypothesis_template is "This text is about{}". We need to change this on a Norwegian model
#For a simple sentiment analysis. Change it to for instance "Sentimentet i denne teksten er {}."

hypothesis_template = "Dette eksempelet handler om {}."

#Some sample categories
candidate_labels = ["positiv", "nøytral", "negativ"]

#Let us pass this to the classifier and allow for multiple labels to be selected
result = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template, multi_label=False)

#Loop through the result
for r in result:
  print(f'\n{r["sequence"]}')
  print(f'{r["scores"][0]:.2f} - {r["labels"][0]}')

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [ ]:
import json

candidate_labels = ["positiv", "nøytral", "negativ"]


with open('./norbench/3polarity.json') as polarity_data:
    polarity_array = json.load(polarity_data)
    
    inputs = [datapoint['text'] for datapoint in polarity_array]
    labels = [datapoint['label'] for datapoint in polarity_array]
    

    print(len(inputs))
    print(len(labels))
    